In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import re
file = open("Episode_II.txt", encoding="utf-8")


script = file.read()

script = script.replace("EXT.", "INT.")
script = script.replace(r'(INT.).*', "INT.")
scenes = script.split("INT.")

# Not a super nice way to do this, but you can also try to use this code in conjunction with get_chars to get more characters in a scene
# It just matches for capital words with spaces in scenes, which unfortunately also gets scene descriptions :(
def get_characters(str):
    chars = set()
    # match = re.findall(r'(\s+\/*)([A-Z]{3,}(\s[A-Z]+)*)', str) 
    match = re.findall(r'(\s+\/*)([A-Z\-]{3,}([^\n]\s[A-Z]+)*)', str)
    if(len(match) > 0):
        for m in match:
            for item in m:
                if(len(item.strip().title()) > 1):
                    chars.add(item.strip().title())
    return chars

def get_chars(scene):
    characters = get_characters(scene)
    nltk_results = ne_chunk(pos_tag(word_tokenize(scene)))
    for nltk_result in nltk_results:
        if type(nltk_result) == Tree and nltk_result.label() == "PERSON":
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            characters.add(name.strip())
    return characters

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\timjh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\timjh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\timjh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [6]:
def resolve_characters(set_people):
    all_people = {
        "Obi-Wan Kenobi": ["Obi-Wan", "Kenobi", "Master Jedi", "Obi-Wan Kenobi", "Master Obi-Wan Kenobi", "Master Kenobi"],
        "Anakin Skywalker": ["boy", "Anakin", "Anakin Skywalker", "Skywalker", "Annie", "my young padawan", "Master Annie", "Master Anakin", "Anne"],
        "Jar Jar Binks": ["Jar Jar", "Binks", "Jar Jar Binks", "Jaja", "Jar"],
        "Nute Gunray": ["Viceroy", "Gunray", "Nute", "Nute Gunray", "Viceroy Gunray"],
        "Padme Amidala": ["Padme Naberrie", "Padme Amidala", "Senator Amidala", "Padm", "Padm Amidala", "Miss Padme", "Amidala"],
        "Yoda": ["Master Yoda", "Yoda"],
        "Mace Windu": ["Master Windu", "Mace", "Windu", "Mace Windu"],
        "Battle Droid": ["battle droid", "battle droids", "OOM-9", "droid sergeant", "battle droid captain", "guard droid", "command droids"],
        "Watto": ["Watto", "Wattos"],
        "Palpatine": ["Chancellor Palpatine", "Darth Sidious", "Sidious", "Sith Lord", "Palpatine", "Chancellor"],
        "Sio Bibble": ["Bibble", "Sio Bibble"],
        "R2-D2": ["Artoo Detoo", "Artoo", "R2-D2", "little blue droid"],
        "C-3PO": ["See Threepio", "Threepio", "C-3PO"],
        "Shmi": ["Shmi", "Mom", "mother", "Shmi Skywalker"],
        "Bail Organa": ["Bail Organa", "Prince Bail Organa"],
        "Ki-Adi-Mundi": ["Ki-Adi-Mundi", "Ki-Adi"],
        "Mas Amedda": ["Vice Chairman Mas Amedda", "Mas Amedda"],
        "Destroyer Droids": ["Droidekas", "Destroyer Droids", "Destroyers"],
        "Cliegg Lars": ["Cliegg Lars", "Cliegg"],
        "Owen Lars": ["Owen", "Owen Lars", "Master Owen"],
        "Beru Lars": ["Beru", "Beru Whitesun"],
        "Sifo-Dyas": ["Master Sifo-Dyas", "Sifo-Dyas", "Sido-Dyas"],
        "Count Dooku": ["Tyranus", "Count Dooku", "Dooku", "Darth Tyranus", "Lord Tyranus"],
        "Jango Fett": ["Jango Fett", "Jango", "Armoured Rocket Man"],
        "Zam Wessel": ["Zam", "Zam Wessel", "Bounty Hunter"],
        "Boba Fett": ["Boba", "Boba Fett"],
        "Tusken Raider": ["Sand People", "Tuskens", "Tusken Raider", "Tusken", "Raider", "Tusken Raiders", "Tusekn Raiders", "Tusken Child"],
        "Dorme": ["Dorme"],
        "Captain Typho": ["Captain Typho", "Typho"],
        "Corde": ["Corde", "Cord"],
        "Ask Aak": ["Senator Ask Aak", "Ask Aak"],
        "Darsana": ["Darsana"],
        "Orn Free Taa": ["Orn Free Taa"],
        "Plo Koon": ["Plo Koon", "Plot Koon"],
        "Royal Guards": ["Royal Guards"],
        "Dar Wac": ["Dar Wac"],
        "Elan Sleazebaggano": ["Elan Sleazebaggano", "Elan"],
        "PK-4": ["PK-4"],
        "Dexter Jettster": ["Dex", "Dexter", "Dexter Jettster"],
        "Queen Jamillia": ["Queen Jamillia", "Your Highness", "Her Highness"],
        "Jocasta Nu": ["Jocasta Nu"],
        "Ryoo": ["Ryoo"],
        "Pooja": ["Pooja"],
        "Sola": ["Sola"],
        "Jobal": ["Jobal Naberrie", "Jobal"],
        "Ruwee": ["Ruwee Naberrie", "Ruwee"],
        "Jedi Child Jack": ["Jedi Child Jack"],
        "Jedi Child May": ["Jedi Child May"],
        "Taun We": ["Taun We"],
        "Lama Su": ["Lama Su"],
        "R4-D17": ["Arfour", "R4 D17", "R4-D17"],
        "Pit Droid": ["Pit Droid"],
        "Clone Troopers": ["Clones", "Clone Trooper", "Clone Captain"],
        "Geonosian": ["Geonosian", "Geonosians"],
        "Poggle the Lesser": ["Poggle", "Poggle the Lesser"],
        "Sun Rit": ["Sun Rit"],
        "Po Nudu": ["Po Nudu"],
        "Tessek": ["Tessek"],
        "Toonbuck Toora": ["Toonbuck Toora"],
        "Shu Mai": ["Shu Mai"],
        "Passel Argente": ["Passel Argente"],
        "Wat Tambor": ["Wat Tambor"],
        "San Hill": ["San Hill"],
        "Super Battle Droids": ["superdroids"],
        "Youngling": ["Youngling"],
        
    }
    return_set = set()
    for item in all_people:
        for thing_string in set_people:
            arr = all_people[item]
            if thing_string in arr:
                return_set.add(item)
    return return_set
    


# name = "Luke"
set_people = {"Obi-Wan Kenobi", "Anakin Skywalker", "Jar Jar Binks", "Nute Gunray", "Padme Amidala", "Yoda", "Mace Windu",\
              "Battle Droid", "Watto", "Palpatine", "Sio Bibble", "R2-D2", "C-3PO", "Shmi", "Bail Organa", "Ki-Adi-Mundi",\
              "Mas Amedda", "Destroyer Droids", "Cliegg Lars", "Owen Lars", "Beru Lars", "Sifo-Dyas", "Count Dooku",\
              "Jango Fett", "Zam Wessel", "Boba Fett", "Tusken Raider", "Dorme", "Captain Typho", "Corde", "Ask Aak", "Darsana",\
              "Orn Free Taa", "Plo Koon", "Royal Guards", "Dar Wac", "Elan Sleazebaggano", "PK-4", "Dexter Jettster",\
              "Queen Jamillia", "Jocasta Nu", "Ryoo", "Pooja", "Sola", "Jobal", "Ruwee", "Jedi Child Jack", "Jedi Child May",\
              "Taun We", "Lama Su", "R4-D17", "Pit Droid", "Clone Troopers", "Geonosian", "Poggle the Lesser", "Sun Rit",\
              "Po Nudu", "Tessek", "Toonbuck Toora", "Shu Mai", "Passel Argente", "Wat Tambor", "San Hill", "Super Battle Droids",\
              "Youngling"}

# print(resolve_characters(set_people))


#print(resolve_characters("Obi-Wan"))

all_chars = set()
all_sets = []
for scene in scenes:
    chars = get_chars(scene)
    if len(chars) != 0:
        cc = resolve_characters(chars)
        print(cc)
        all_chars |= cc
        all_sets.append(cc)


{'Padme Amidala', 'Count Dooku'}
set()
{'Corde', 'Captain Typho', 'Jar Jar Binks', 'Padme Amidala', 'R2-D2'}
set()
{'Padme Amidala', 'Dorme', 'Palpatine', 'Darsana'}
set()
{'Yoda', 'Count Dooku', 'Palpatine', 'Padme Amidala', 'Obi-Wan Kenobi', 'Mace Windu'}
set()
{'Jar Jar Binks', 'Anakin Skywalker'}
{'Captain Typho', 'Jar Jar Binks', 'Dorme', 'Anakin Skywalker', 'Padme Amidala', 'Obi-Wan Kenobi'}
set()
{'Mace Windu', 'Yoda'}
{'R2-D2'}
{'R2-D2', 'Captain Typho', 'Anakin Skywalker'}
{'R2-D2'}
{'Anakin Skywalker'}
{'R2-D2', 'Anakin Skywalker'}
set()
{'Dorme', 'Captain Typho', 'Anakin Skywalker'}
set()
{'Anakin Skywalker'}
{'Zam Wessel', 'Yoda', 'Anakin Skywalker'}
{'Zam Wessel', 'Anakin Skywalker'}
{'Elan Sleazebaggano', 'Zam Wessel', 'Anakin Skywalker'}
{'Zam Wessel', 'Anakin Skywalker'}
{'Yoda', 'Jar Jar Binks', 'Dorme', 'Anakin Skywalker', 'Padme Amidala', 'Mace Windu'}
set()
{'Captain Typho', 'Dorme', 'Anakin Skywalker', 'Padme Amidala', 'Obi-Wan Kenobi', 'R2-D2'}
set()
set()
{'Obi-W

In [4]:
print(all_chars)

{'Ryoo', 'Yoda', 'Nute Gunray', 'Tusken Raider', 'Darsana', 'Ruwee', 'Poggle the Lesser', 'Mace Windu', 'Shmi', 'Pooja', 'C-3PO', 'Corde', 'Captain Typho', 'Sio Bibble', 'Sifo-Dyas', 'Jar Jar Binks', 'Palpatine', 'Elan Sleazebaggano', 'Queen Jamillia', 'Sola', 'Jobal', 'Zam Wessel', 'Geonosian', 'R2-D2', 'Watto', 'Lama Su', 'Jango Fett', 'Boba Fett', 'Beru Lars', 'Dexter Jettster', 'Mas Amedda', 'Cliegg Lars', 'Count Dooku', 'Dorme', 'Clone Troopers', 'Anakin Skywalker', 'Padme Amidala', 'Obi-Wan Kenobi', 'R4-D17', 'Owen Lars'}


In [5]:
from collections import defaultdict

def def_value():
    return 0

all_connections = defaultdict(def_value)
all_people = defaultdict(def_value)

def split_set(data):
    d_list = list(data)
    return list([frozenset((a, b)) for idx, a in enumerate(d_list) for b in d_list[idx + 1:]])

# Returns a defaultdict of type set:int representing two characters in a set with the number of their occurrances together,
# given a list of sets
def group_chars(data):
    for item in data:
        item = frozenset(item)
        if len(item) > 2:
            subsets = split_set(item)
            for subset in subsets:
                all_connections[subset] += 1
        if len(item) == 2:
            all_connections[item] += 1

# Returns a dictionary of string:int representing how many scenes a character is in
def count_chars(data):
    for item in data:
        for char in item:
            all_people[char] += 1

group_chars(all_sets)
count_chars(all_sets)

print(all_people)

#print(all_connections) # all_connections holds a key of two characters as a frozenset, and a value of their occurrances together in scenes

defaultdict(<function def_value at 0x0000012E1B988820>, {'Padme Amidala': 25, 'Count Dooku': 19, 'Corde': 1, 'Captain Typho': 5, 'Jar Jar Binks': 6, 'R2-D2': 18, 'Dorme': 5, 'Palpatine': 7, 'Darsana': 1, 'Yoda': 14, 'Obi-Wan Kenobi': 11, 'Mace Windu': 13, 'Anakin Skywalker': 58, 'Zam Wessel': 4, 'Elan Sleazebaggano': 1, 'Shmi': 6, 'Dexter Jettster': 1, 'Sio Bibble': 1, 'Nute Gunray': 3, 'Queen Jamillia': 1, 'Ryoo': 1, 'Pooja': 1, 'Ruwee': 3, 'Sola': 2, 'Jobal': 2, 'Lama Su': 1, 'Sifo-Dyas': 1, 'Jango Fett': 14, 'Clone Troopers': 4, 'R4-D17': 3, 'Boba Fett': 9, 'Watto': 1, 'C-3PO': 8, 'Beru Lars': 6, 'Cliegg Lars': 3, 'Owen Lars': 4, 'Tusken Raider': 5, 'Mas Amedda': 1, 'Geonosian': 5, 'Poggle the Lesser': 2})
